# Vibration Environment

We can use the IMU to determine what the vibration specrum was like during the flight. Looking at all axises of the accelerometer in the IMU we see the plots get significantly noisy during the boost portion of the flight.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, sqrt
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]
acc_x, acc_y, acc_z = columns[6:9]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

accel = []
for i, t in enumerate(timestamp):
    accel.append(sqrt((acc_x[i]*acc_x[i]) + (acc_y[i]*acc_y[i]) + (acc_z[i]*acc_z[i])))

ax = plt.figure(figsize=(16,6))
plt.title(r"IMU Acceleration Magnitude")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp, acc_x, alpha=0.75, label="X (Up)")
plt.plot(timestamp, acc_y, alpha=0.6, label="Y")
plt.plot(timestamp, acc_z, alpha=0.6, label="Z")
ax.axes[0].set_xlim([-5,42])
plt.legend()
plt.show()

In [ ]:
ax = plt.figure(figsize=(16,6))
plt.title(r"IMU Acceleration Magnitude")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp, accel, alpha=0.75, label="Up")
ax.axes[0].set_xlim([-5,42])
#plt.legend()
plt.show()

If we take a Fourier transform of the acceleration magnitude data then we can see the spectral power over many frequencies during the flight.

In [ ]:
from scipy import signal

fs = 819.2
f, Pxx_den = signal.periodogram(divide(accel, 9.8), fs,  scaling='spectrum')

ax = plt.figure(figsize=(16,6))
plt.title(r"IMU Acceleration Power Spectral Density")
plt.ylabel(r"PSD [g${}^2$/Hz]")
plt.xlabel(r"Frequency [Hz]")
plt.semilogy(f[1:], Pxx_den[1:], alpha=0.75, label="Up")
ax.axes[0].set_xlim([-1,fs/2])
ax.axes[0].set_ylim([10e-14,1])
#plt.legend()
plt.show()

## Waterfall Sonogram

The most interesting thing is to look at the change in vibration power spectrum over time. We use a waterfall chart to do this with sliding windows:

In [ ]:
from matplotlib.colors import LogNorm
from matplotlib import cm
import numpy as np
from matplotlib import gridspec


fs = 819.2
data = accel
fft_size = 1024
overlap_fac = 0.75

hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
pad_end_size = fft_size          # the last segment can overlap the end of the data array by no more than one window size
total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
t_max = len(data) / np.float32(fs)

window = np.hanning(fft_size)  # our half cosine window
inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size

proc = np.concatenate((data, np.zeros(pad_end_size)))              # the data to process
result = np.empty((total_segments, fft_size), dtype=np.float32)    # space to hold the result

for i in xrange(total_segments):                      # for each segment
    current_hop = hop_size * i                        # figure out the current segment offset
    segment = proc[current_hop:current_hop+fft_size]  # get the current segment
    windowed = segment * window                       # multiply by the half cosine function
    padded = np.append(windowed, inner_pad)           # add 0s to double the length of the data
    spectrum = np.fft.fft(padded) / fft_size          # take the Fourier Transform and scale by the number of samples
    autopower = np.abs(spectrum * np.conj(spectrum))  # find the autopower spectrum
    result[i, :] = autopower[:fft_size]               # append to the results array

result = 20*np.log10(result)          # scale to db
result = np.clip(result, -200, 10)    # clip values

fig = plt.figure(figsize=(16,8))
plt.subplots_adjust(wspace=0.001)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 7]) 

ax1 = plt.subplot(gs[0])
plt.title(r"")
plt.ylabel(r"Time [s]")
plt.xlabel(r"Acceleration [g]")
ax1.plot(divide(accel, 9.8), timestamp, 'k-', alpha=0.6, lw=0.4)
plt.xlim([-0.5,12])
plt.ylim([timestamp[-1]+0.75,timestamp[0]])

ax2 = plt.subplot(gs[1])
plt.title(r"IMU Acceleration Power Spectrum")
plt.xlabel(r"Vibration Frequency [Hz]]")
img = ax2.imshow(result, origin='upper', cmap='plasma', interpolation='bicubic', aspect='auto')
tics = range(0,401,50)
plt.xticks([t*2.5 for t in tics], tics)
ax2.tick_params(axis='x', colors='white', labelcolor='black', length=10)
ax2.tick_params(axis='y', colors='white', labelcolor='black', length=10)
ticklabels = ax2.get_yticklabels()
plt.setp(ticklabels, visible=False)

cbaxes = fig.add_axes([0.91, 0.125, 0.02, 0.775]) 
cbar = plt.colorbar(img, cax = cbaxes)
cbar.ax.set_ylabel('Vibration Power [dB]')

plt.show()